# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
# import pyodbc
import os
import warnings

# Constants

In [2]:
# Define the years for each dataframe
years = [2017, 2018, 2019, 2021, 2022]

In [3]:
def safe_convert(val):
    try:
        return int(val)
    except ValueError:
        print(f"Value {val} can't be converted to int")
        return None
    
def import_mdb(MDBs, DRV, PWD, NAMES):
    
    databases = {}
    
    for MDB, NAME in zip(MDBs, NAMES):
        # connect to db
        con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB,PWD))
        cur = con.cursor()

        # List all tables in the database
        tables = list(map(lambda t: t.table_name, con.cursor().tables(tableType='TABLE')))

        # Initialize an empty dictionary to hold your dataframes and databases
        database = {}

        # Try to read each table one by one
        for table in tables:
            try:
                df = pd.read_sql(f'SELECT * FROM [{table}]', con)  # enclose table name in brackets
                database[table] = df
                print(f"Successfully read table: {table}")
            except Exception as e:
                print(f"Failed to read table: {table}")
                print(f"Error: {e}")
        databases[NAME] = database
        
    return databases

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', 'pandas only support SQLAlchemy connectable.*')
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
# warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

# Import Enrollment Data

In [4]:
# Enroll Data Filepaths
ENROLL_PATHS = [
               '../../data/enrollment_2018/ENROLL2018.csv',
               '../../data/enrollment_2019/ENROLL2019.csv',
               '../../data/enrollment_2021/ENROLL2021.csv',
               '../../data/enrollment_2022/ENROLL2022.csv',
              ]
ENROLL_NAMES = [
               'ENROLL2018',
               'ENROLL2019',
               'ENROLL2021',
               'ENROLL2022',
              ]

# data16 = None
data17 = None
data18 = pd.read_csv(ENROLL_PATHS[0])
data19 = pd.read_csv(ENROLL_PATHS[1])
data21 = pd.read_csv(ENROLL_PATHS[2])
data22 = pd.read_csv(ENROLL_PATHS[3])

# data16 = data18[data18["YEAR"] == 2016]
data17 = data18[data18["YEAR"] == 2017]
data18 = data18[data18["YEAR"] == 2018]
data19 = data19[data19["YEAR"] == 2019]
data21 = data21[data21["YEAR"] == 2021]
data22 = data22[data22["YEAR"] == 2022]

enrollment_dataframes = [data17, data18, data19, data21, data22]

In [5]:
first_set_cols = [
'schoolcode',
"districtcode",
"countycode",
 'year',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'state'
      ]

for i, dataframe in enumerate(enrollment_dataframes):
    enrollment_dataframes[i] = enrollment_dataframes[i][~enrollment_dataframes[i]['ENTITY_CD'].astype(str).str.endswith('0000')]
    enrollment_dataframes[i]['ENTITY_CD'] = enrollment_dataframes[i]['ENTITY_CD'].apply(safe_convert)
    enrollment_dataframes[i]['schoolcode'] = enrollment_dataframes[i]['ENTITY_CD']
    enrollment_dataframes[i]['black'] = enrollment_dataframes[i]['PER_BLACK']
    enrollment_dataframes[i]['white'] = enrollment_dataframes[i]['PER_WHITE']
    enrollment_dataframes[i]['hispanic'] = enrollment_dataframes[i]['PER_HISP']
    enrollment_dataframes[i]['lowincome'] = enrollment_dataframes[i]['PER_ECDIS']
    enrollment_dataframes[i]["state"] = "nyc"
    enrollment_dataframes[i]["schoolcode"] = enrollment_dataframes[i]["schoolcode"].astype(str)
    enrollment_dataframes[i]["schoolcode"] = enrollment_dataframes[i]["schoolcode"].str.zfill(9)
    enrollment_dataframes[i]["countycode"] = enrollment_dataframes[i]["schoolcode"].str[:3].astype(str)
    enrollment_dataframes[i]["districtcode"] = enrollment_dataframes[i]["schoolcode"].str[:6].astype(str)
    enrollment_dataframes[i]['districtcode'] = enrollment_dataframes[i]['districtcode'].astype(str)
    enrollment_dataframes[i]['countycode'] = enrollment_dataframes[i]['countycode'].astype(str)

    enrollment_dataframes[i]["districtcode"] = enrollment_dataframes[i]["districtcode"] + enrollment_dataframes[i]["state"]
    enrollment_dataframes[i]["countycode"] = enrollment_dataframes[i]["countycode"] + enrollment_dataframes[i]["state"]
    enrollment_dataframes[i]["year"] = years[i]
    enrollment_dataframes[i] = enrollment_dataframes[i][first_set_cols]

/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1871447998.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enrollment_dataframes[i]['ENTITY_CD'] = enrollment_dataframes[i]['ENTITY_CD'].apply(safe_convert)
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1871447998.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enrollment_dataframes[i]['schoolcode'] = enrollment_dataframes[i]['ENTITY_CD']
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1871447998.p

In [6]:
enrollment_df = pd.concat(enrollment_dataframes)
enrollment_df = enrollment_df.reset_index(drop=True)

In [7]:
enrollment_df

,schoolcode,districtcode,countycode,year,lowincome,white,black,hispanic,state
0,22902040001,229020nyc,229nyc,2017,62,96,0,2,nyc
1,22902040002,229020nyc,229nyc,2017,67,98,0,1,nyc
2,30101060001,301010nyc,301nyc,2017,49,93,2,2,nyc
3,30101060003,301010nyc,301nyc,2017,36,92,1,4,nyc
4,30101060004,301010nyc,301nyc,2017,40,93,1,3,nyc
...,...,...,...,...,...,...,...,...,...
23675,680601060001,680601nyc,680nyc,2022,47,90,1,5,nyc
23676,680601060002,680601nyc,680nyc,2022,56,93,0,3,nyc
23677,680601060005,680601nyc,680nyc,2022,59,91,0,5,nyc
23678,680801040001,680801nyc,680nyc,2022,60,92,1,3,nyc


# Import Dropout Rates

In [8]:
data22 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2022.csv", thousands=',')
data21 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2021.csv", thousands=',')
data19 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2019.csv", thousands=',')
data18 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2018.csv", thousands=',')
data17 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2017.csv", thousands=',')
# data16 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2016.csv", thousands=',')

dropout_dfs = [data17, data18, data19, data21, data22]

# Combine Dropout Data

In [9]:
# Remove districts, only keep schools
for i, df in enumerate(dropout_dfs):
    dropout_dfs[i].columns = dropout_dfs[i].columns.str.lower()
    dropout_dfs[i] = df[df['aggregation_type'] == 'School']

#   Only keep schools which are present in all years    #
#########################################################

# Convert the 'ID' column of each DataFrame to a set
set1 = set(dropout_dfs[0]['aggregation_code'])
set2 = set(dropout_dfs[1]['aggregation_code'])
set3 = set(dropout_dfs[2]['aggregation_code'])
set4 = set(dropout_dfs[3]['aggregation_code'])
set5 = set(dropout_dfs[4]['aggregation_code'])
# set6 = set(dropout_dfs[5]['aggregation_code'])

# Find the intersection of all 4 sets - i.e., the common IDs
common_ids = set1 & set2 & set3 & set4 & set5

# Filter each DataFrame to only include rows with a common ID
for i, df in enumerate(dropout_dfs):
    dropout_dfs[i] = df[df['aggregation_code'].isin(common_ids)]
    
###########################################################

common_ids = set(df['aggregation_code'])

# Initialize a list to store the updated dataframes
updated_dfs = []

# Iterate over the dropout dataframes and the years together
for year, df in zip(years, dropout_dfs):
    # Add a new column 'year' to the dataframe
    df['year'] = year
    # Append the updated dataframe to the list
    updated_dfs.append(df)

# Concatenate the updated dataframes together
dropout_df = pd.concat(updated_dfs)

# drop disttricts from the dataframe
dropout_df = dropout_df[~dropout_df['aggregation_code'].astype(str).str.endswith('0000.0')]

# Reset the index of the combined dataframe
dropout_df = dropout_df.reset_index(drop=True)

common_ids = set(dropout_df['aggregation_code'])

In [10]:
dropout_df = dropout_df[dropout_df["membership_desc"].str.contains("4 Year Outcome", na=False)]
dropout_df = dropout_df[~dropout_df["membership_desc"].str.contains("August", na=False)]
dropout_df = dropout_df[dropout_df["subgroup_name"].str.lower() == "all students"]
dropout_df["schoolcode"] = dropout_df["aggregation_code"].astype(str)
dropout_df["report_school_year"] = dropout_df["report_school_year"].astype(str)
dropout_df["year"] = "20" + dropout_df["report_school_year"].str[-2:]
dropout_df["year"] = dropout_df["year"].astype(int)
dropout_df["dropout"] =  dropout_df["dropout_pct"].str.replace('%', '', regex=False).replace('-', '0', regex=False).astype(float)
dropout_df = dropout_df[["year", "schoolcode", "dropout"]]

In [11]:
dropout_df

,year,schoolcode,dropout
34,2017,10100010034,17.0
91,2017,10100860907,0.0
141,2017,10100860960,3.0
194,2017,10201040001,3.0
249,2017,10306060008,3.0
...,...,...,...
538895,2022,670401040001,3.0
538997,2022,671201060002,4.0
539099,2022,671501040002,8.0
539208,2022,680601060001,10.0


In [12]:
dropout_df.drop_duplicates(inplace=True)

In [13]:
enrollment_dropout = dropout_df.merge(enrollment_df, on=['schoolcode', 'year'], how='inner')

In [14]:
enrollment_dropout

,year,schoolcode,dropout,districtcode,countycode,lowincome,white,black,hispanic,state
0,2017,10100010034,17.0,101000nyc,101nyc,55,20,53,15,nyc
1,2017,10100860907,0.0,101008nyc,101nyc,60,3,76,14,nyc
2,2017,10100860960,3.0,101008nyc,101nyc,73,8,73,12,nyc
3,2017,10201040001,3.0,102010nyc,102nyc,37,97,1,1,nyc
4,2017,10306060008,3.0,103060nyc,103nyc,13,86,3,4,nyc
...,...,...,...,...,...,...,...,...,...,...
6133,2022,670401040001,3.0,670401nyc,670nyc,40,96,2,1,nyc
6134,2022,671201060002,4.0,671201nyc,671nyc,51,89,0,7,nyc
6135,2022,671501040002,8.0,671501nyc,671nyc,45,92,1,3,nyc
6136,2022,680601060001,10.0,680601nyc,680nyc,47,90,1,5,nyc


# Import Total Enrollment Data

In [15]:
# Enroll Data Filepaths
TOTAL_PATHS = [
               '../../data/enrollment_2018/TOTAL2018.csv',
               '../../data/enrollment_2019/TOTAL2019.csv',
               '../../data/enrollment_2021/TOTAL2021.csv',
               '../../data/enrollment_2022/TOTAL2022.csv',
              ]

# data16 = None
data17 = None
data18 = pd.read_csv(TOTAL_PATHS[0])
data19 = pd.read_csv(TOTAL_PATHS[1])
data21 = pd.read_csv(TOTAL_PATHS[2])
data22 = pd.read_csv(TOTAL_PATHS[3])

# data16 = data18[data18["YEAR"] == 2016]
data17 = data18[data18["YEAR"] == 2017]
data18 = data18[data18["YEAR"] == 2018]
data19 = data19[data19["YEAR"] == 2019]
data21 = data21[data21["YEAR"] == 2021]
data22 = data22[data22["YEAR"] == 2022]

total_enrollment_dataframes = [data17, data18, data19, data21, data22]

In [16]:
total_enrollment_dataframes[0]

,ENTITY_CD,ENTITY_NAME,YEAR,PK,PKHALF,PKFULL,KHALF,KFULL,1,2,3,4,5,6,7,8,9,10,11,12,UGE,UGS,K12
0,1,NYC Public Schools,2017,71970,2406,69564,0,69741,73282,73874,73022,74043,70478,67761,66243,67288,79984,76607,67411,64358,10031,10586,944709
1,2,Large Cities,2017,8682,1091,7591,0,8081,8347,8585,8488,8425,7879,7485,7238,7360,9023,7812,6187,6591,594,882,102977
2,3,High Need/Resource Urban-Suburban Districts,2017,12807,6538,6269,0,16369,16434,16678,16651,16218,15783,14968,14732,14849,16683,16141,14357,14401,933,1368,206565
3,4,High Need/Resource Rural Districts,2017,6945,3215,3730,0,11061,10747,10914,11153,11007,11061,10710,11037,10955,11299,11302,10609,10662,419,756,143692
9,5,Average Need Districts,2017,19476,14154,5322,1467,50297,52594,53512,54730,55676,56402,56087,56216,58239,59482,59338,58481,58898,2268,3936,737623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15855,680601060002,PENN YAN MIDDLE SCHOOL,2017,0,0,0,0,0,0,0,0,0,0,90,111,113,0,0,0,0,0,0,314
15856,680601060005,PENN YAN ELEMENTARY SCHOOL,2017,31,0,31,0,115,94,121,94,89,96,0,0,0,0,0,0,0,1,0,610
15857,680801040000,DUNDEE CENTRAL SCHOOL DISTRICT,2017,65,17,48,0,52,42,66,48,49,34,52,44,55,53,57,41,50,2,5,650
15858,680801040001,DUNDEE JUNIOR-SENIOR HIGH SCHOOL,2017,0,0,0,0,0,0,0,0,0,0,0,44,55,53,57,40,50,0,1,300


In [17]:
first_set_cols = [
'schoolcode',
"totalenrolled",
 'year',
      ]

for i, dataframe in enumerate(total_enrollment_dataframes):
    total_enrollment_dataframes[i] = total_enrollment_dataframes[i][~total_enrollment_dataframes[i]['ENTITY_CD'].astype(str).str.endswith('0000')]
    total_enrollment_dataframes[i]['ENTITY_CD'] = total_enrollment_dataframes[i]['ENTITY_CD'].apply(safe_convert)
    total_enrollment_dataframes[i]['schoolcode'] = total_enrollment_dataframes[i]['ENTITY_CD']
    total_enrollment_dataframes[i]['schoolcode'] = total_enrollment_dataframes[i]['schoolcode'].astype(str)
    total_enrollment_dataframes[i]['totalenrolled'] = (total_enrollment_dataframes[i]['9'] + 
                                               total_enrollment_dataframes[i]['10'] +
                                               total_enrollment_dataframes[i]['11'] +
                                               total_enrollment_dataframes[i]['12'] 
                                               )

    total_enrollment_dataframes[i]["year"] =  total_enrollment_dataframes[i]["YEAR"]
    total_enrollment_dataframes[i] = total_enrollment_dataframes[i][first_set_cols]

/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1907416550.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_enrollment_dataframes[i]['ENTITY_CD'] = total_enrollment_dataframes[i]['ENTITY_CD'].apply(safe_convert)
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1907416550.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_enrollment_dataframes[i]['schoolcode'] = total_enrollment_dataframes[i]['ENTITY_CD']
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipyke

In [18]:
total_enrollment_dataframe = pd.concat(total_enrollment_dataframes)

In [19]:
total_enrollment_dataframe

,schoolcode,totalenrolled,year
0,1,288360,2017
1,2,29613,2017
2,3,61582,2017
3,4,43872,2017
9,5,236199,2017
...,...,...,...
16584,680601060001,383,2022
16585,680601060002,0,2022
16586,680601060005,0,2022
16588,680801040001,183,2022


In [20]:
enrollment_dropout_total = enrollment_dropout.merge(total_enrollment_dataframe, on=['schoolcode', 'year'], how='inner')

In [21]:
enrollment_dropout_total

,year,schoolcode,dropout,districtcode,countycode,lowincome,white,black,hispanic,state,totalenrolled
0,2017,10100010034,17.0,101000nyc,101nyc,55,20,53,15,nyc,2451
1,2017,10100860907,0.0,101008nyc,101nyc,60,3,76,14,nyc,353
2,2017,10100860960,3.0,101008nyc,101nyc,73,8,73,12,nyc,394
3,2017,10201040001,3.0,102010nyc,102nyc,37,97,1,1,nyc,260
4,2017,10306060008,3.0,103060nyc,103nyc,13,86,3,4,nyc,1612
...,...,...,...,...,...,...,...,...,...,...,...
6133,2022,670401040001,3.0,670401nyc,670nyc,40,96,2,1,nyc,255
6134,2022,671201060002,4.0,671201nyc,671nyc,51,89,0,7,nyc,234
6135,2022,671501040002,8.0,671501nyc,671nyc,45,92,1,3,nyc,258
6136,2022,680601060001,10.0,680601nyc,680nyc,47,90,1,5,nyc,383


# Import Virtual Mode Data

In [22]:
virtual = pd.read_csv("../../data/New_York_Schools_LearningModelData_Final.csv", thousands=',')
virtual['Charter'] = virtual['Charter'].replace({'Yes': 1, 'No': 0})

In [23]:
virtual = virtual[virtual['TimePeriodStart'].str.endswith(('21', '22'))]
virtual.head()

,StateName,StateAbbrev,DataLevel,Charter,SchoolName,SchoolType,NCESSchoolID,StateAssignedSchoolID,DistrictName,DistrictType,NCESDistrictID,StateAssignedDistrictID,TimePeriodInterval,TimePeriodStart,TimePeriodEnd,EnrollmentTotal,LearningModel,LearningModelGrK5,LearningModelGr68,LearningModelGr912,LearningModelStateCat,LearningModelStateCatGrK5,LearningModelStateCatGr68,LearningModelStateCatGr912,EnrollmentInPerson,EnrollmentHybrid,EnrollmentVirtual,StaffCount,StaffCountInPerson
12,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/3/21,1/9/21,378,Virtual,NaN,NaN,NaN,Remote Only,NaN,NaN,NaN,79.0,0.0,299.0,81.0,45.0
13,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/10/21,1/16/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,237.0,141.0,81.0,81.0
14,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/17/21,1/23/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,226.0,152.0,81.0,76.0
15,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/24/21,1/30/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,229.0,149.0,81.0,81.0
16,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/31/21,2/6/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,230.0,148.0,81.0,80.0


In [24]:
# Convert the date columns to datetime format.
virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])

# Create a new column for year
virtual['YEAR'] = virtual['TimePeriodStart'].dt.year

# Fill in any missing values in LearningModel with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].fillna('InPerson')

# Replace 'In-person' with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].replace('In-person', 'InPerson')

# Calculate the number of days for each row
virtual['Days'] = (virtual['TimePeriodEnd'] - virtual['TimePeriodStart']).dt.days

# Group by School, Year, LearningModel, and Charter and sum the number of days
grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel', 'Charter'])['Days'].sum().reset_index()

# Pivot the data so we have separate columns for each learning model
pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR', 'Charter'], columns='LearningModel', values='Days', fill_value=0)

# # Group by School, Year, and LearningModel and sum the number of days
# grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel'])['Days'].sum().reset_index()

# # Pivot the data so we have separate columns for each learning model
# pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR'], columns='LearningModel', values='Days', fill_value=0)

# Reset the index
pivot.reset_index(inplace=True)

# Calculate the total days in each year
pivot['TotalDays'] = pivot['Virtual'] + pivot['Hybrid'] + pivot['InPerson']

# Calculate the percentage of days that are virtual and hybrid for each year
pivot['VirtualPercent'] = pivot['Virtual'] / pivot['TotalDays']
pivot['HybridPercent'] = pivot['Hybrid'] / pivot['TotalDays']

# Calculate the score for each year
pivot['Score'] = (pivot['Virtual'] + 0.5 * pivot['Hybrid']) / pivot['TotalDays']

# Reset the column names after pivot
pivot.columns.name = None

/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1507681578.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
/var/folders/8y/bjy80p890fg6hgk57b8wz5d40000gn/T/ipykernel_15117/1507681578.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])


In [25]:
pivot = pivot.drop(columns=['InPerson', 'Hybrid', 'Virtual', 'TotalDays'])
pivot
pivot["schoolcode"] = pivot["StateAssignedSchoolID"].astype(str)
pivot["year"] = pivot["YEAR"]
pivot["virtualper"] = pivot["VirtualPercent"]
pivot["hybridper"] = pivot["HybridPercent"]
pivot["schoolmode"] = pivot["Score"]
pivot["charter"] = pivot["Charter"]
pivot = pivot[["schoolcode", "schoolmode", "virtualper", "schoolmode", "hybridper", "charter"]]

pivot

,schoolcode,schoolmode,virtualper,schoolmode,hybridper,charter
0,10100010014,0.48,0.04,0.48,0.88,0
1,10100010016,0.48,0.04,0.48,0.88,0
2,10100010018,0.48,0.04,0.48,0.88,0
3,10100010019,0.48,0.04,0.48,0.88,0
4,10100010023,0.48,0.04,0.48,0.88,0
...,...,...,...,...,...,...
4398,680601060001,0.08,0.08,0.08,0.00,0
4399,680601060002,0.08,0.08,0.08,0.00,0
4400,680601060005,0.08,0.08,0.08,0.00,0
4401,680801040001,0.08,0.08,0.08,0.00,0


In [26]:
export_dataframe = enrollment_dropout_total.merge(pivot, on = "schoolcode")
export_dataframe.loc[export_dataframe["year"] != 2021, ["hybridper", "virtualper", "schoolmode"]] = 0
export_dataframe["schoolcode"] = export_dataframe["schoolcode"] + "nyc"

export_dataframe = export_dataframe[export_dataframe['totalenrolled'] != 0]

# Identify the unique years in the dataset.
unique_years = set(export_dataframe['year'])

# Group the data by ENTITY_CD and get the unique years for each group.
entity_groups = export_dataframe.groupby('schoolcode')['year'].unique().reset_index()

# Find the ENTITY_CD values which have all the unique years.
valid_entity_cd = entity_groups[entity_groups['year'].apply(lambda x: set(x) == unique_years)]['schoolcode']

# Filter the main data for these ENTITY_CD values.
export_dataframe = export_dataframe[export_dataframe['schoolcode'].isin(valid_entity_cd)]
export_dataframe.shape

(5825, 16)

In [27]:
export_dataframe

,year,schoolcode,dropout,districtcode,countycode,lowincome,white,black,hispanic,state,totalenrolled,schoolmode,virtualper,schoolmode,hybridper,charter
0,2017,10100010034nyc,17.0,101000nyc,101nyc,55,20,53,15,nyc,2451,0.00,0.00,0.00,0.00,0
1,2018,10100010034nyc,18.0,101000nyc,101nyc,64,22,51,15,nyc,2548,0.00,0.00,0.00,0.00,0
2,2019,10100010034nyc,17.0,101000nyc,101nyc,66,22,50,15,nyc,2470,0.00,0.00,0.00,0.00,0
3,2021,10100010034nyc,6.0,101000nyc,101nyc,66,20,51,17,nyc,2524,0.48,0.04,0.48,0.88,0
4,2022,10100010034nyc,9.0,101000nyc,101nyc,68,18,49,19,nyc,2510,0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5844,2017,680801040001nyc,16.0,680801nyc,680nyc,61,95,1,2,nyc,200,0.00,0.00,0.00,0.00,0
5845,2018,680801040001nyc,4.0,680801nyc,680nyc,63,97,1,2,nyc,203,0.00,0.00,0.00,0.00,0
5846,2019,680801040001nyc,7.0,680801nyc,680nyc,60,94,0,3,nyc,198,0.00,0.00,0.00,0.00,0
5847,2021,680801040001nyc,9.0,680801nyc,680nyc,60,94,0,2,nyc,179,0.08,0.08,0.08,0.00,0


In [28]:
export_dataframe = export_dataframe[export_dataframe["year"] != 2022]
export_dataframe.describe()

,year,dropout,lowincome,white,black,hispanic,totalenrolled,schoolmode,virtualper,schoolmode,hybridper,charter
count,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.000000,4660.0
mean,2018.750000,5.802146,57.143562,49.137768,17.596352,24.494206,651.138841,0.120747,0.044189,0.120747,0.153116,0.0
std,1.479179,6.516042,24.987977,38.986114,22.212875,25.140561,658.053424,0.229908,0.120200,0.229908,0.295218,0.0
min,2017.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,2017.750000,1.000000,39.000000,5.000000,1.000000,4.000000,269.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,2018.500000,4.000000,57.000000,57.000000,7.000000,14.000000,422.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,2019.500000,8.000000,80.000000,90.000000,27.000000,43.250000,817.750000,0.000000,0.000000,0.000000,0.000000,0.0
max,2021.000000,66.000000,100.000000,100.000000,93.000000,100.000000,5943.000000,0.880000,0.840000,0.880000,1.000000,0.0


In [29]:
export_dataframe

,year,schoolcode,dropout,districtcode,countycode,lowincome,white,black,hispanic,state,totalenrolled,schoolmode,virtualper,schoolmode,hybridper,charter
0,2017,10100010034nyc,17.0,101000nyc,101nyc,55,20,53,15,nyc,2451,0.00,0.00,0.00,0.00,0
1,2018,10100010034nyc,18.0,101000nyc,101nyc,64,22,51,15,nyc,2548,0.00,0.00,0.00,0.00,0
2,2019,10100010034nyc,17.0,101000nyc,101nyc,66,22,50,15,nyc,2470,0.00,0.00,0.00,0.00,0
3,2021,10100010034nyc,6.0,101000nyc,101nyc,66,20,51,17,nyc,2524,0.48,0.04,0.48,0.88,0
5,2017,10201040001nyc,3.0,102010nyc,102nyc,37,97,1,1,nyc,260,0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,2021,680601060001nyc,7.0,680601nyc,680nyc,47,91,0,4,nyc,408,0.08,0.08,0.08,0.00,0
5844,2017,680801040001nyc,16.0,680801nyc,680nyc,61,95,1,2,nyc,200,0.00,0.00,0.00,0.00,0
5845,2018,680801040001nyc,4.0,680801nyc,680nyc,63,97,1,2,nyc,203,0.00,0.00,0.00,0.00,0
5846,2019,680801040001nyc,7.0,680801nyc,680nyc,60,94,0,3,nyc,198,0.00,0.00,0.00,0.00,0


In [30]:
export_dataframe.to_csv("../final_data_component/final_data_nyc_dropout.csv")